In [1]:
import sys
import os
import glob
import re 
import pandas as pd
import numpy as np
from lxml import etree
import matplotlib.pyplot as plt
import requests
import time
from collections import Counter


In [2]:
columns = ["bert_cased", "bert_multilingual_cased", "mt5_base", "xlm_roberta_base"]

In [6]:
df = pd.read_csv("./../data/translations_tokenized/Dios Habla Hoy (DHH).tsv", sep="\t", index_col=0)

In [12]:
df

,Unnamed: 0.1,id,text,bert_multilingual_cased,bert_cased,mt5_base,xlm_roberta_base,codebook,text_count_tokens,characters_count,...,bert_multilingual_cased_count_parts_tokens,bert_multilingual_cased_count_unks,mt5_base_count_tokens,mt5_base_count_starting_tokens,mt5_base_count_parts_tokens,mt5_base_count_unks,xlm_roberta_base_count_tokens,xlm_roberta_base_count_starting_tokens,xlm_roberta_base_count_parts_tokens,xlm_roberta_base_count_unks
0,0,b.1CH.001.001,"Adán, Set, Enós,","Ad|##án|,|Set|,|En|##ós|,","Ad|##án|,|Set|,|En|##ós|,","▁A|dán|,|▁Set|,|▁E|nós|,","▁A|dán|,|▁Set|,|▁En|ós|,",1CH,6,17,...,2,0,26,4,0,0,26,4,0,0
1,1,b.1CH.001.010,"Cus fue el padre de Nimrod, el primer hombre p...","Cu|##s|fue|el|padre|de|Ni|##m|##rod|,|el|prime...","C|##us|f|##ue|el|pad|##re|de|Ni|##m|##rod|,|el...","▁Cu|s|▁fue|▁el|▁padre|▁de|▁Nim|rod|,|▁el|▁prim...","▁Cu|s|▁fue|▁el|▁padre|▁de|▁Nim|rod|,|▁el|▁prim...",1CH,15,68,...,3,1,88,14,0,1,86,14,0,1
2,2,b.1CH.001.011,"De Misraim descienden los ludeos, los anameos,...",De|Mis|##rai|##m|des|##cie|##nden|los|lu|##deo...,De|Mi|##s|##rai|##m|des|##cie|##nden|los|l|##u...,"▁De|▁Mis|raim|▁des|ci|enden|▁los|▁|lude|os|,|▁...","▁De|▁Mis|ra|im|▁des|ci|enden|▁los|▁lu|de|os|,|...",1CH,15,78,...,14,0,107,12,0,0,108,12,0,0
3,3,b.1CH.001.012,"los patruseos, los casluhítas y los caftoritas...","los|patru|##se|##os|,|los|cas|##lu|##hí|##tas|...","los|pat|##rus|##eo|##s|,|los|ca|##s|##lu|##h|#...","▁los|▁patru|se|os|,|▁los|▁ca|sluh|í|tas|▁|y|▁l...","▁los|▁patru|se|os|,|▁los|▁cas|luh|í|tas|▁y|▁lo...",1CH,15,85,...,12,0,115,13,0,0,113,13,0,0
4,4,b.1CH.001.013,"Canaán fue padre de Sidón, su primer hijo, y d...","Can|##a|##án|fue|padre|de|Sid|##ón|,|su|primer...","Can|##a|##án|f|##ue|pad|##re|de|Sid|##ón|,|su|...","▁Cana|án|▁fue|▁padre|▁de|▁Si|dón|,|▁su|▁primer...","▁Cana|án|▁fue|▁padre|▁de|▁Si|dón|,|▁su|▁primer...",1CH,14,53,...,3,0,72,12,0,0,70,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30748,30748,b.ZEP.003.005,Pero el Señor está en la ciudad; \nél hace lo ...,Pero|el|Señor|está|en|la|ciudad|;|él|hace|lo|b...,Per|##o|el|Se|##ño|##r|est|##á|en|la|c|##iu|##...,▁Pero|▁el|▁S|eñor|▁está|▁en|▁la|▁ciudad|;|▁él|...,▁Pero|▁el|▁Señor|▁está|▁en|▁la|▁ciudad|;|▁él|▁...,ZEP,38,164,...,6,0,209,31,0,0,201,31,0,0
30749,30749,b.ZEP.003.006,"Dice el Señor: \n«He destruido naciones, \nhe ...",Di|##ce|el|Señor|:|«|He|des|##tru|##ido|nacion...,Di|##ce|el|Se|##ño|##r|:|«|He|des|##tr|##uid|#...,▁Di|ce|▁el|▁S|eñor|:|▁«|He|▁destru|ido|▁na|cio...,▁Di|ce|▁el|▁Señor|:|▁«|He|▁destru|ido|▁na|cion...,ZEP,41,204,...,11,0,257,35,0,0,253,35,0,0
30750,30750,b.ZEP.003.007,Pensé: “Así Jerusalén me temerá \ny aceptará q...,Pen|##sé|:|[UNK]|Así|Jerusalén|me|teme|##rá|y|...,Pen|##s|##é|:|“|As|##í|Je|##rus|##al|##én|me|t...,▁Pens|é|:|▁“|A|s|í|▁Jerusal|én|▁me|▁teme|rá|▁|...,▁Pen|sé|:|▁“|A|sí|▁Jer|usa|lén|▁me|▁teme|rá|▁y...,ZEP,35,185,...,14,6,237,31,0,0,230,31,0,0
30751,30751,b.ZEP.003.008,"Por eso, espérenme ustedes el día \nen que me ...","Por|eso|,|es|##pé|##ren|##me|us|##tede|##s|el|...","Po|##r|es|##o|,|es|##p|##é|##ren|##me|us|##ted...","▁Por|▁eso|,|▁|e|spér|en|me|▁|usted|es|▁el|▁día...","▁Por|▁eso|,|▁es|pé|ren|me|▁ustedes|▁el|▁día|▁e...",ZEP,59,270,...,18,0,350,51,0,0,338,51,0,0


In [22]:
df[["id", "text","codebook"] + columns]

,id,text,codebook,bert_cased,bert_multilingual_cased,mt5_base,xlm_roberta_base
0,b.1CH.001.001,"Adán, Set, Enós,",1CH,"Ad|##án|,|Set|,|En|##ós|,","Ad|##án|,|Set|,|En|##ós|,","▁A|dán|,|▁Set|,|▁E|nós|,","▁A|dán|,|▁Set|,|▁En|ós|,"
1,b.1CH.001.010,"Cus fue el padre de Nimrod, el primer hombre p...",1CH,"C|##us|f|##ue|el|pad|##re|de|Ni|##m|##rod|,|el...","Cu|##s|fue|el|padre|de|Ni|##m|##rod|,|el|prime...","▁Cu|s|▁fue|▁el|▁padre|▁de|▁Nim|rod|,|▁el|▁prim...","▁Cu|s|▁fue|▁el|▁padre|▁de|▁Nim|rod|,|▁el|▁prim..."
2,b.1CH.001.011,"De Misraim descienden los ludeos, los anameos,...",1CH,De|Mi|##s|##rai|##m|des|##cie|##nden|los|l|##u...,De|Mis|##rai|##m|des|##cie|##nden|los|lu|##deo...,"▁De|▁Mis|raim|▁des|ci|enden|▁los|▁|lude|os|,|▁...","▁De|▁Mis|ra|im|▁des|ci|enden|▁los|▁lu|de|os|,|..."
3,b.1CH.001.012,"los patruseos, los casluhítas y los caftoritas...",1CH,"los|pat|##rus|##eo|##s|,|los|ca|##s|##lu|##h|#...","los|patru|##se|##os|,|los|cas|##lu|##hí|##tas|...","▁los|▁patru|se|os|,|▁los|▁ca|sluh|í|tas|▁|y|▁l...","▁los|▁patru|se|os|,|▁los|▁cas|luh|í|tas|▁y|▁lo..."
4,b.1CH.001.013,"Canaán fue padre de Sidón, su primer hijo, y d...",1CH,"Can|##a|##án|f|##ue|pad|##re|de|Sid|##ón|,|su|...","Can|##a|##án|fue|padre|de|Sid|##ón|,|su|primer...","▁Cana|án|▁fue|▁padre|▁de|▁Si|dón|,|▁su|▁primer...","▁Cana|án|▁fue|▁padre|▁de|▁Si|dón|,|▁su|▁primer..."
...,...,...,...,...,...,...,...
30748,b.ZEP.003.005,Pero el Señor está en la ciudad; \nél hace lo ...,ZEP,Per|##o|el|Se|##ño|##r|est|##á|en|la|c|##iu|##...,Pero|el|Señor|está|en|la|ciudad|;|él|hace|lo|b...,▁Pero|▁el|▁S|eñor|▁está|▁en|▁la|▁ciudad|;|▁él|...,▁Pero|▁el|▁Señor|▁está|▁en|▁la|▁ciudad|;|▁él|▁...
30749,b.ZEP.003.006,"Dice el Señor: \n«He destruido naciones, \nhe ...",ZEP,Di|##ce|el|Se|##ño|##r|:|«|He|des|##tr|##uid|#...,Di|##ce|el|Señor|:|«|He|des|##tru|##ido|nacion...,▁Di|ce|▁el|▁S|eñor|:|▁«|He|▁destru|ido|▁na|cio...,▁Di|ce|▁el|▁Señor|:|▁«|He|▁destru|ido|▁na|cion...
30750,b.ZEP.003.007,Pensé: “Así Jerusalén me temerá \ny aceptará q...,ZEP,Pen|##s|##é|:|“|As|##í|Je|##rus|##al|##én|me|t...,Pen|##sé|:|[UNK]|Así|Jerusalén|me|teme|##rá|y|...,▁Pens|é|:|▁“|A|s|í|▁Jerusal|én|▁me|▁teme|rá|▁|...,▁Pen|sé|:|▁“|A|sí|▁Jer|usa|lén|▁me|▁teme|rá|▁y...
30751,b.ZEP.003.008,"Por eso, espérenme ustedes el día \nen que me ...",ZEP,"Po|##r|es|##o|,|es|##p|##é|##ren|##me|us|##ted...","Por|eso|,|es|##pé|##ren|##me|us|##tede|##s|el|...","▁Por|▁eso|,|▁|e|spér|en|me|▁|usted|es|▁el|▁día...","▁Por|▁eso|,|▁es|pé|ren|me|▁ustedes|▁el|▁día|▁e..."


In [23]:
df["text"].head(5).str.count("[ \´\`¶\(»\]\?\.\–\!’•\|“\>\)\-\—\:\}\*\&…¿\/=¡\"\'·+\{\#\[;­,«~]")+1

0     6
1    15
2    15
3    15
4    14
Name: text, dtype: int64

In [24]:
df["text"].head(5).str.len()+1


0    17
1    68
2    78
3    85
4    53
Name: text, dtype: int64

In [25]:
df["bert_multilingual_cased"].str.count("(\|##|▁)")

0         2
1         3
2        14
3        12
4         3
         ..
30748     6
30749    11
30750    14
30751    18
30752     6
Name: bert_multilingual_cased, Length: 30753, dtype: int64

In [26]:
df["bert_multilingual_cased"].str.count("\|")+1

0         8
1        18
2        29
3        27
4        17
         ..
30748    44
30749    52
30750    50
30751    77
30752    32
Name: bert_multilingual_cased, Length: 30753, dtype: int64

In [28]:
for path in glob.glob("./../data/translations_tokenized/*.tsv"):
    print(path)

    input_name_str  = os.path.splitext(os.path.split(path)[1])[0]

    df = pd.read_csv(path, sep="\t", index_col=0)[["id", "text","codebook"] + columns]

    df["codebook"] = df.id.str[2:5] 


    df["text_count_tokens"] = df["text"].str.count("[ \´\`¶\(»\]\?\.\–\!’•\|“\>\)\-\—\:\}\*\&…¿\/=¡\"\'·+\{\#\[;­,«~]")+1
    df["characters_count"] = df["text"].str.len()+1


    for column in columns:


        df[column + "_count_tokens"] = df[column].str.count("\|")+1

        df[column + "_count_parts_tokens"] = df[column].str.count("(\|##|▁)")

        df[column + "_count_unks"] = df[column].str.count("\[UNK\]")

    df.to_csv("./../data/translations_tokenized_counts/" + input_name_str + ".tsv", sep="\t")


./../data/translations_tokenized\Biblia del Jubileo (JBS).tsv
./../data/translations_tokenized\Dios Habla Hoy (DHH).tsv
./../data/translations_tokenized\La Biblia de las Américas (LBLA).tsv
./../data/translations_tokenized\La Palabra (España) (BLP).tsv
./../data/translations_tokenized\La Palabra (Hispanoamérica) (BLPH).tsv
./../data/translations_tokenized\Nueva Biblia de las Américas (NBLA).tsv
./../data/translations_tokenized\Nueva Biblia Viva (NBV).tsv
./../data/translations_tokenized\Nueva Traducción Viviente (NTV).tsv
./../data/translations_tokenized\Nueva Versión Internacional (Castilian) (CST).tsv
./../data/translations_tokenized\Nueva Versión Internacional (NVI).tsv
./../data/translations_tokenized\Palabra de Dios para Todos (PDT).tsv
./../data/translations_tokenized\Reina Valera Actualizada (RVA-2015).tsv
./../data/translations_tokenized\Reina Valera Contemporánea (RVC).tsv
./../data/translations_tokenized\Reina Valera Revisada (RVR1977).tsv
./../data/translations_tokenized\Rei

C:\Users\calvotello\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./../data/translations_tokenized\SF_2011-07-10_FRE_BDP_(BIBLE DES PEUPLES).tsv
./../data/translations_tokenized\SF_2012-01-07_ARA_ARANAV_(NEW ARABIC VERSION (KETAB EL HAYAT)).tsv
./../data/translations_tokenized\SF_2012-01-07_AZB_AZERI_(DESCRIPTION_EN=SOUTHERN AZERI TRANSLATION).tsv
./../data/translations_tokenized\SF_2012-01-07_FRE_FREPGR_(BIBLE PERRET-GENTIL ET RILLIET).tsv
./../data/translations_tokenized\SF_2012-01-27_NOR_NORSMB_(STUDENTMÅLLAGSBIBELEN FRÅ 1921).tsv
./../data/translations_tokenized\SF_2012-03-16_FRE_VIGOUROUX_(BIBLE VIGOUROUX 1902).tsv
./../data/translations_tokenized\SF_2012-04-21_HEB_OSMHB_(OPEN SCRIPTURES MORPHOLOGICAL HEBREW BIBLE).tsv
./../data/translations_tokenized\SF_2012-08-08_ENG_OEB_(OPEN ENGLISH BIBLE (US SPELLING)).tsv
./../data/translations_tokenized\SF_2012-08-14_GER_LUT_1545_LH_(LUTHER 1545 (LETZTE HAND)).tsv
./../data/translations_tokenized\SF_2013-04-12_FRE_MACH2012_(LA BIBLE MACHAIRA).tsv
./../data/translations_tokenized\SF_2013-04-19_FRE_BO1877_(

In [29]:
df

,id,text,codebook,bert_cased,bert_multilingual_cased,mt5_base,xlm_roberta_base,text_count_tokens,characters_count,bert_cased_count_tokens,...,bert_cased_count_unks,bert_multilingual_cased_count_tokens,bert_multilingual_cased_count_parts_tokens,bert_multilingual_cased_count_unks,mt5_base_count_tokens,mt5_base_count_parts_tokens,mt5_base_count_unks,xlm_roberta_base_count_tokens,xlm_roberta_base_count_parts_tokens,xlm_roberta_base_count_unks
0,b.1CH.001.004,"1-4 Los descendientes de Adán son: \nSet, \nEn...",1CH,1|-|4|Los|descend|##ient|##es|de|Ad|##án|son|:...,"1|-|4|Los|descendientes|de|Ad|##án|son|:|Set|,...",▁1-4|▁Los|▁descend|ientes|▁de|▁A|dán|▁son|:|▁S...,▁1-4|▁Los|▁descend|ientes|▁de|▁A|dán|▁son|:|▁S...,37,162,63,...,0,54,15,0,53,23,0,55,23,0
1,b.1CH.001.011,Éstos son los pueblos que descienden de Misrai...,1CH,É|##sto|##s|son|los|pu|##eb|##los|que|des|##ci...,És|##tos|son|los|pueblos|que|des|##cie|##nden|...,▁É|stos|▁son|▁los|▁|puebl|os|▁que|▁des|ci|ende...,▁És|tos|▁son|▁los|▁pueblo|s|▁que|▁des|ci|enden...,21,109,40,...,0,36,15,0,37,16,0,37,16,0
2,b.1CH.001.012,"los patruseos, \nlos casluhítas \ny los caftor...",1CH,"los|pat|##rus|##eo|##s|,|los|ca|##s|##lu|##h|#...","los|patru|##se|##os|,|los|cas|##lu|##hí|##tas|...","▁los|▁patru|se|os|,|▁los|▁ca|sluh|í|tas|▁|y|▁l...","▁los|▁patru|se|os|,|▁los|▁cas|luh|í|tas|▁y|▁lo...",16,95,35,...,0,31,15,0,32,13,0,31,13,0
3,b.1CH.001.013,"Canaán tuvo dos hijos: \nSidón, \nHet.",1CH,Can|##a|##án|t|##u|##vo|dos|hi|##jos|:|Sid|##ó...,"Can|##a|##án|tuvo|dos|hijos|:|Sid|##ón|,|Het|.","▁Cana|án|▁|tuvo|▁dos|▁hij|os|:|▁Si|dón|,|▁Het|.","▁Cana|án|▁tuvo|▁dos|▁hijos|:|▁Si|dón|,|▁Het|.",9,37,16,...,0,12,3,0,13,6,0,11,6,0
4,b.1CH.001.016,-16 Éstos son los pueblos que descienden de Ca...,1CH,-|16|É|##sto|##s|son|los|pu|##eb|##los|que|des...,-|16|És|##tos|son|los|pueblos|que|des|##cie|##...,▁|-16|▁É|stos|▁son|▁los|▁|puebl|os|▁que|▁des|c...,▁|-16|▁És|tos|▁son|▁los|▁pueblo|s|▁que|▁des|ci...,37,184,69,...,0,60,22,0,64,28,0,62,28,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26649,b.ZEP.003.005,¡Esos malvados no tienen vergüenza! \n»Yo esto...,ZEP,¡|E|##so|##s|ma|##l|##va|##dos|no|tie|##nen|ve...,¡|Es|##os|mal|##vado|##s|no|tienen|ver|##g|##ü...,▁¡|Es|os|▁mal|vados|▁no|▁tiene|n|▁ver|güen|za|...,▁¡|Es|os|▁mal|vado|s|▁no|▁tienen|▁ver|gü|enza|...,32,147,58,...,0,43,11,0,47,24,0,42,24,0
26650,b.ZEP.003.006,"»Yo he destruido naciones, \ny he derribado su...",ZEP,"»|Yo|he|des|##tr|##uid|##o|na|##cion|##es|,|y|...","»|Yo|he|des|##tru|##ido|naciones|,|y|he|der|##...","▁»|Yo|▁he|▁destru|ido|▁na|ciones|,|▁|y|▁he|▁de...","▁»|Yo|▁he|▁destru|ido|▁na|ciones|,|▁y|▁he|▁der...",35,176,64,...,0,45,10,0,52,29,0,46,29,0
26651,b.ZEP.003.007,"»Todo esto lo hice por ti, Jerusalén. \nPensé ...",ZEP,"»|To|##do|est|##o|lo|hi|##ce|p|##or|t|##i|,|Je...","»|Todo|esto|lo|hi|##ce|por|ti|,|Jerusalén|.|Pe...","▁»|Todo|▁esto|▁lo|▁|hice|▁por|▁ti|,|▁Jerusal|é...","▁»|To|do|▁esto|▁lo|▁hic|e|▁por|▁ti|,|▁Jer|usa|...",34,171,65,...,0,45,11,0,50,29,0,45,29,0
26652,b.ZEP.003.008,"»Y ahora, como han actuado así, \nya se acerca...",ZEP,"»|Y|ah|##ora|,|com|##o|ha|##n|act|##uad|##o|as...","»|Y|ahora|,|como|han|act|##uado|así|,|ya|se|ac...","▁»|Y|▁|ahora|,|▁como|▁han|▁actua|do|▁|así|,|▁y...","▁»|Y|▁ahora|,|▁como|▁han|▁actua|do|▁así|,|▁ya|...",66,282,107,...,0,82,16,0,90,55,0,78,55,0


In [30]:
df.columns.tolist()

['id',
 'text',
 'codebook',
 'bert_cased',
 'bert_multilingual_cased',
 'mt5_base',
 'xlm_roberta_base',
 'text_count_tokens',
 'characters_count',
 'bert_cased_count_tokens',
 'bert_cased_count_parts_tokens',
 'bert_cased_count_unks',
 'bert_multilingual_cased_count_tokens',
 'bert_multilingual_cased_count_parts_tokens',
 'bert_multilingual_cased_count_unks',
 'mt5_base_count_tokens',
 'mt5_base_count_parts_tokens',
 'mt5_base_count_unks',
 'xlm_roberta_base_count_tokens',
 'xlm_roberta_base_count_parts_tokens',
 'xlm_roberta_base_count_unks']